In [1]:
from pyspark.sql.functions import *

In [2]:
df = sc.parallelize([
    (10,'',10000),(20,'Female',30000),(None,'Male',80000),(None,'Male',5000)
]).toDF(['age','gender','income'])

In [3]:
df.show()

+----+------+------+
| age|gender|income|
+----+------+------+
|  10|      | 10000|
|  20|Female| 30000|
|null|  Male| 80000|
|null|  Male|  5000|
+----+------+------+



In [4]:
df.describe().show()

+-------+------------------+------+-----------------+
|summary|               age|gender|           income|
+-------+------------------+------+-----------------+
|  count|                 2|     4|                4|
|   mean|              15.0|  null|          31250.0|
| stddev|7.0710678118654755|  null|34247.87098005753|
|    min|                10|      |             5000|
|    max|                20|  Male|            80000|
+-------+------------------+------+-----------------+



In [6]:
avg_age = df.na.drop().agg(avg('age')).collect()[0][0]

In [7]:
print(avg_age)

15.0


In [8]:
sparkf_replaceNull = udf(lambda x : avg_age if x == None else x)

In [9]:
no_null_df = df.withColumn('age',sparkf_replaceNull(col('age')))
no_null_df.show()

+----+------+------+
| age|gender|income|
+----+------+------+
|  10|      | 10000|
|  20|Female| 30000|
|15.0|  Male| 80000|
|15.0|  Male|  5000|
+----+------+------+



In [11]:
treat_missing = udf(lambda x : 'Male_assume' if x=="" else x)
no_missing_df = no_null_df.withColumn('new_gender',treat_missing(no_null_df.gender))
no_missing_df.show()

+----+------+------+-----------+
| age|gender|income| new_gender|
+----+------+------+-----------+
|  10|      | 10000|Male_assume|
|  20|Female| 30000|     Female|
|15.0|  Male| 80000|       Male|
|15.0|  Male|  5000|       Male|
+----+------+------+-----------+



In [13]:
no_outlier_df = no_missing_df.filter(col('income')>=10000)
no_outlier_df.show()

+----+------+------+-----------+
| age|gender|income| new_gender|
+----+------+------+-----------+
|  10|      | 10000|Male_assume|
|  20|Female| 30000|     Female|
|15.0|  Male| 80000|       Male|
+----+------+------+-----------+

